In [1]:
import os
import cv2
import math
import numpy as np
import pandas as pd
import mediapipe as mp
import argparse
import moviepy.editor
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def calculate_3d(point1, point2, point3):
    # Calculate the vectors between the left hand, elbow, and shoulder landmarks
    point_1_2 = [(point2[i] - point1[i]) for i in range(3)]
    point_2_3 = [(point2[i] - point3[i]) for i in range(3)]

    # Calculate the dot product and the magnitudes of the vectors
    dot_product = sum([point_1_2[i] * point_2_3[i] for i in range(3)])
    point_1_2_mag = math.sqrt(sum([coord**2 for coord in point_1_2]))
    point_2_3_mag = math.sqrt(sum([coord**2 for coord in point_2_3]))

    # Calculate the angle between the left hand, elbow, and shoulder landmarks in degrees
    angle = math.degrees(math.acos(dot_product / (point_1_2_mag * point_2_3_mag)))

    return angle

In [3]:
# 분류 model load
class_model = tf.keras.models.load_model('./model/classify_model.h5')
# 32*99 dataframe
df_cor = pd.read_csv('./data/coordinate.csv')
point_groups = [
('LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST'),
('LEFT_HIP', 'LEFT_SHOULDER', 'LEFT_ELBOW'),
('LEFT_KNEE', 'LEFT_HIP', 'LEFT_SHOULDER'),
('LEFT_ANKLE', 'LEFT_KNEE', 'LEFT_HIP'),
('RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST'),
('RIGHT_HIP', 'RIGHT_SHOULDER', 'RIGHT_ELBOW'),
('RIGHT_KNEE', 'RIGHT_HIP', 'RIGHT_SHOULDER'),
('RIGHT_ANKLE', 'RIGHT_KNEE', 'RIGHT_HIP')
]
# 열 이름 리스트 만들기
column_names = [f'{group[0]}_ANGLE' for group in point_groups]
# 각 그룹의 각도 계산 및 데이터프레임에 추가
for group, col_name in zip(point_groups, column_names):
    df_cor[col_name] = df_cor.apply(lambda row: calculate_3d(
        [row[f'{group[0]}.x'], row[f'{group[0]}.y'], row[f'{group[0]}.z']],
        [row[f'{group[1]}.x'], row[f'{group[1]}.y'], row[f'{group[1]}.z']],
        [row[f'{group[2]}.x'], row[f'{group[2]}.y'], row[f'{group[2]}.z']]
    ), axis=1)
X = df_cor.iloc[:,-8:]
sequence_length = 32  # 시퀀스 길이 설정
Xsequence = []
for i in range(0, len(X) - sequence_length + 1, sequence_length):
    Xsequence.append(X[i:i+sequence_length])
Xsequence = np.array(Xsequence)
y_pred = class_model.predict(Xsequence)

y_pred_class = np.argmax(y_pred, axis=1)

1/1 [==============================] - 2s 2s/step


In [4]:
y_pred_class

array([2], dtype=int64)